<a href="https://colab.research.google.com/github/Lotses/GB_API/blob/main/Lesson4_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt install mongodb > log

In [2]:
!service mongodb start

 * Starting database mongodb
   ...done.


In [3]:
! pip install pymongo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [64]:
from pymongo import MongoClient
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import re
from pprint import pprint

In [10]:
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None  # default='warn'

In [5]:
client = MongoClient()

In [59]:
db = client.db_test

In [11]:
response = requests.get('https://rskrf.ru/ratings/produkty-pitaniya')
response

<Response [200]>

In [12]:
soup = bs(response.content, 'html.parser')
tag_li = soup.find_all('div', class_='h5 card-title with-text')
result = pd.DataFrame({"Наименование": [i.text for i in tag_li]})

In [47]:
test_df = pd.DataFrame(columns=['общий рейтинг', 'безопасность', 'качество'])
test_df
# циклом пробегаемся по всем продуктам и обрабатываем каждую ссылку

for link in tag_li:
    # создаём адрес продукта
    href = 'https://rskrf.ru/' + link.find('a').get('href')
    response_2 = requests.get(href)
    # создаём дом
    dom = bs(response_2.content, 'html.parser')
    # ищем позиции рейтинга
    tag_2 = dom.find_all('div', class_='rating-item')
    try:
        tag_3 = dom.find_all(string=re.compile('Пищевая ценность'))[1]
    except:
        pass
    indexes = []
    var_df = pd.DataFrame([{'общий рейтинг': '', 'безопасность': '', 'качество': '', 'ценность': ''}])
    if tag_3:
        var_df['ценность'].values[0] = tag_3
    else:
        pass
    for count,item in enumerate(tag_2):
        if count == 13:
            break
        else:
            pos = item.find_all('span')[0].text.lower().strip()
            if pos in ['общий рейтинг', 'безопасность', 'качество']:
                var = item.find_all('span')[1].text
                if var:
                  var_df[pos].values[0] = var
                else:
                  var_df[pos].values[0] = 0.0
            else:
                pass
    test_df = test_df.append(var_df, ignore_index = True )
result_df = pd.concat([result, test_df], sort=False, axis=1)
result_df

,Наименование,общий рейтинг,безопасность,качество,ценность
0,Братья Чебурашкины Молоко питьевое пастеризова...,5.0000,5.000,5.000,"Пищевая ценность 100 г продукта: жир от 3,6 г ..."
1,Му-у Молоко цельное отборное питьевое пастериз...,5.0000,5.000,5.000,Пищевая ценность продукта в 100 г: жира - от 3...
2,Снежок Сметана 20% «Снежок»,5.0000,5.000,5.000,\nПищевая ценность соответствует установленным...
3,Молоко Оренбуржья Молоко питьевое пастеризован...,5.0000,5.000,5.000,Пищевая ценность (содержание в 100 г продукта)...
4,"Коровка из Кореновки Сметана, массовая доля жи...",5.0000,5.000,,\nПищевая ценность соответствует установленным...
5,Агрокомплекс Выселковский Сметана с массовой д...,5.0000,5.000,,\nПищевая ценность соответствует установленным...
6,"Лебедянь молоко Сметана, массовая доля жира 20...",5.0000,5.000,,\nПищевая ценность соответствует установленным...
7,Любимое молоко Молоко питьевое пастеризованное...,5.0000,5.000,5.000,Пищевая ценность продукта (средние значения) н...
8,Молоко Брянского молочного комбината Молоко пи...,5.0000,5.000,5.000,Пищевая ценность в 100г продукта (средние знач...
9,Сударыня Молоко питьевое ультрапастеризованн...,5.0000,5.000,5.000,"Пищевая ценность в 100 г продукта: жиры - 3,2 ..."


In [21]:
a = int(input('Введите минимальную оценку качества : '))
b = int(input('Введите минимальную общую оценку : '))

Введите минимальную оценку качества : 4
Введите минимальную общую оценку : 4


In [50]:
result_df['общий рейтинг'] = result_df['общий рейтинг'].astype(float)
result_df['качество'] = result_df['качество'].replace('', 0.0).astype(float)



In [60]:
result_df_2 = result_df.query("`общий рейтинг` > @b | `качество` > @a")
result_df_2

,Наименование,общий рейтинг,безопасность,качество,ценность
0,Братья Чебурашкины Молоко питьевое пастеризова...,5.0,5.000,5.0,"Пищевая ценность 100 г продукта: жир от 3,6 г ..."
1,Му-у Молоко цельное отборное питьевое пастериз...,5.0,5.000,5.0,Пищевая ценность продукта в 100 г: жира - от 3...
2,Снежок Сметана 20% «Снежок»,5.0,5.000,5.0,\nПищевая ценность соответствует установленным...
3,Молоко Оренбуржья Молоко питьевое пастеризован...,5.0,5.000,5.0,Пищевая ценность (содержание в 100 г продукта)...
4,"Коровка из Кореновки Сметана, массовая доля жи...",5.0,5.000,0.0,\nПищевая ценность соответствует установленным...
5,Агрокомплекс Выселковский Сметана с массовой д...,5.0,5.000,0.0,\nПищевая ценность соответствует установленным...
6,"Лебедянь молоко Сметана, массовая доля жира 20...",5.0,5.000,0.0,\nПищевая ценность соответствует установленным...
7,Любимое молоко Молоко питьевое пастеризованное...,5.0,5.000,5.0,Пищевая ценность продукта (средние значения) н...
8,Молоко Брянского молочного комбината Молоко пи...,5.0,5.000,5.0,Пищевая ценность в 100г продукта (средние знач...
9,Сударыня Молоко питьевое ультрапастеризованн...,5.0,5.000,5.0,"Пищевая ценность в 100 г продукта: жиры - 3,2 ..."


In [66]:
db.db_test.insert_many(result_df_2.to_dict('records'))
for doc in db.db_test.find():
  pprint(doc)

{'_id': ObjectId('63fb8e8d2cec1cee94d55d5e'),
 'Наименование': 'Братья Чебурашкины Молоко питьевое пастеризованное цельное, '
                 'массовая доля жира от 3,6% до 4,6%, «Братья Чебурашкины»',
 'безопасность': '5.000',
 'качество': 5.0,
 'общий рейтинг': 5.0,
 'ценность': 'Пищевая ценность 100 г продукта: жир от 3,6 г до 4,6 г, белок '
             '3,0 г, углеводы 4,7 г; энергетическая ценность (калорийность) от '
             '264 кДж (63 ккал) до 301 кДж (72 ккал) ; М.Д.Ж. 3,7%'}
{'_id': ObjectId('63fb8e8d2cec1cee94d55d5f'),
 'Наименование': 'Му-у Молоко цельное отборное питьевое пастеризованное, '
                 'массовая доля жира от 3,4% до 6,0%, «Му-у» ',
 'безопасность': '5.000',
 'качество': 5.0,
 'общий рейтинг': 5.0,
 'ценность': 'Пищевая ценность продукта в 100 г: жира - от 3,4 до 6,0 г, '
             'белка- 2,8 г, углеводов - 4,7 г. Энергетическая ценность - от 61 '
             'ккал/255 кДж до 84 ккал/352 кДж (фактическое значение м.д.ж. '
             '3,4